In [46]:
import re
import pandas as pd
import numpy as np

### Extraction

In [3]:
# Extract raw data from .webgateway
with open('./aktaion/data/proxyData//benignData/BlueCoat_Large.webgateway','rb') as f:
    data = f.readlines()

In [148]:
# Extract the user agent
user_ag = [str(i).split('"')[11] for i in data]
series_user_ag = pd.Series(user_ag,name = 'rawUserAg')
# Format the user agent accordingly to its standard components
pat_user_ag = re.compile('(?P<browser>\w+)\/(?P<version>[0-9\.]*)(?P<systemInfo> \((?<=\().*?\))?(?P<platform> [^(]*(?P<platformDetails> \([^()]*?\))??)?(?P<platform2> [^(]*(?P<platformDetails2> \([^()]*?\))??)?(?P<extensions> [^(]*)?$')
dt_user_ag = series_user_ag.str.extract(pat_user_ag, expand = False)
# Format strings
for col in dt_user_ag.columns:
    dt_user_ag.loc[:,col] = dt_user_ag.loc[:,col].str.strip(' ()').str.lower()
# Fill null values
dt_user_ag.fillna('not_available', inplace = True)

In [156]:
series_user_ag[10]

'Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/20100101 Firefox/5.0 CometBird/5.0'

In [149]:
dt_user_ag.head()

,browser,version,systemInfo,platform,platformDetails,platform2,platformDetails2,extensions
0,mozilla,5.0,windows; u; windows nt 5.1; en-us,"applewebkit/534.10 (khtml, like gecko","khtml, like gecko",chrome/8.0.552.237 safari/534.10,not_available,not_available
1,mozilla,5.0,windows nt 5.1; rv:2.0.1,gecko/20100101 firefox/4.0.1,not_available,not_available,not_available,not_available
2,mozilla,5.0,ipod; u; cpu iphone os 4_1 like mac os x; en-us,"applewebkit/532.9 (khtml, like gecko","khtml, like gecko",version/4.0.5 mobile/8b117 safari/6531.22.7,not_available,not_available
3,mozilla,5.0,windows nt 5.1; rv:5.0,gecko/20100101 firefox/5.0,not_available,not_available,not_available,not_available
4,not_available,not_available,not_available,not_available,not_available,not_available,not_available,not_available


### Feature engineering
3 clear challenges stand out:
* Categorical data
* Cardinality
* Variable number of attributes (systemInfo, platformDetails, extensions)

We will process feature by feature

#### Browser

In [6]:
dt_user_ag.browser.value_counts()

mozilla           1248012
not_available      141772
opera               22897
syncer               1690
applecoremedia       1467
                   ...   
blackberry9000          1
mikrotik                1
libtorrent              1
arcadehoops             1
phonebook               1
Name: browser, Length: 135, dtype: int64

> * High cardinality
> * Number falls quickly

In [7]:
browser_version_count = dt_user_ag.groupby('browser')['version'].value_counts()
browser_version_count.sort_values(ascending=False)

browser          version      
mozilla          5.0              744021
                 4.0              503693
not_available    not_available    141772
opera            9.80              21592
applecoremedia   1.0.0.8            1467
                                   ...  
libtorrent       0.14.1.0              1
makeup           9.0                   1
mediaplayer      0000                  1
mikrotik         3.                    1
101photofilters  4.0.0                 1
Name: version, Length: 234, dtype: int64

> * Low version cardinality relative to the browser

##### Encoding
To deal with cardinality, I suggest a method based on the estimator encoding:
* Principle: replace a category by the value associated with one if its features
* Target variable: as we do not have relevant features (numerical) we will create are own: the relative frequency (lvl_0)
* Relative feature: the relative frequency of the browser version is calculated relative to the same browser versions only (lvl_1)

In [24]:
# Mapping dicts
browser_lvl_0_dict = dt_user_ag.browser.value_counts(normalize = True).to_dict()
browser_lvl_1_dict = dt_user_ag.groupby('browser')['version'].value_counts(normalize = True).to_dict()

In [98]:
def map_tuple(x, mapping, tup):
    not_av = ('not_available','not_available')
    return mapping.get((x[tup[0]],x[tup[1]]),not_av)

dt_user_ag['browser_lvl_0'] = dt_user_ag.loc[:,'browser'].map(browser_lvl_0_dict)
dt_user_ag['browser_lvl_1'] = dt_user_ag.loc[:,['browser','version']].apply(lambda x: map_tuple(x,browser_lvl_1_dict,('browser','version')),axis = 1)

### System Info
Besides the challenges shared with the former case, the System Info introduces a new characteristic.
Indeed,the number of system info available per record is  not fixed. We suggest the following method to encode it:
* Encode each system info and its version following the same steps as previously (estimator encoding)
* Summarize the multiple system info at the record level with a set of descriptive statistics (mean, min, max, number)

First, we need to build the mapping system info - frequency

In [66]:
# Extract system infos
system_info_arr = dt_user_ag.systemInfo.str.split(';',expand=True).values.ravel()
# EXtract system info details (version)
pattern_system_info = re.compile('^(?P<name>[a-z.]*[ .0-9]*\w*)[ :/]?(?P<version>[0-9.]*)')
system_info_ser = pd.Series(system_info_arr).str.strip()
system_info_levels = system_info_ser.loc[np.invert(system_info_ser.isna())].str.extract(pattern_system_info).fillna('not_available')
system_info_levels.version = system_info_levels.version.where(system_info_levels.version != '','not_available')

In [100]:
# Mappings dicts
system_info_lvl_0_dict = system_info_levels.name.value_counts(normalize = True)
system_info_lvl_1_dict = system_info_levels.groupby('name')['version'].value_counts(normalize = True).to_dict()
# Extract and unpivot system info per record
dt_user_ag_sys_info = dt_user_ag.systemInfo.str.split(';',expand=True)
dt_user_ag_sys_info = dt_user_ag_sys_info.reset_index().melt(id_vars = 'index',value_vars = dt_user_ag_sys_info.columns,value_name = 'system_info').loc[:,['index','system_info']]
dt_user_ag_sys_info = dt_user_ag_sys_info.dropna()
# Extract details
dt_user_ag_sys_info_levels = dt_user_ag_sys_info.system_info.str.strip().str.extract(pattern_system_info).fillna('not_available')
dt_user_ag_sys_info_levels.version = dt_user_ag_sys_info_levels.version.where(dt_user_ag_sys_info_levels.version != '','not_available')
# Map values
dt_user_ag_sys_info_levels['system_info_lvl_0'] = dt_user_ag_sys_info_levels.loc[:,'name'].map(system_info_lvl_0_dict)
dt_user_ag_sys_info_levels['system_info_lvl_1'] = dt_user_ag_sys_info_levels.loc[:,['name','version']].apply(lambda x: map_tuple(x,system_info_lvl_1_dict,('name','version')),axis = 1)

In [103]:
# Aggregate values
dt_user_ag_sys_info_levels = dt_user_ag_sys_info_levels.merge(dt_user_ag_sys_info.loc[:,'index'], left_index = True, right_index = True)
desc_stat = ['mean','max','min']
apply_map = {'system_info_lvl_0':desc_stat+['count'],
            'system_info_lvl_1':desc_stat}
dt_user_ag_sys_info_levels = dt_user_ag_sys_info_levels.groupby('index')['system_info_lvl_0','system_info_lvl_1'].agg(apply_map)
dt_user_ag_sys_info_levels.columns = ['_'.join(col) for col in dt_user_ag_sys_info_levels.columns]

In [112]:
# Append values to dataset
dt_user_ag = dt_user_ag.merge(dt_user_ag_sys_info_levels, right_index = True, left_index = True, how = 'left')

In [116]:
dt_user_ag.system_info_lvl_0_count = dt_user_ag.system_info_lvl_0_count.where(dt_user_ag.systemInfo != 'not_available',dt_user_ag.system_info_lvl_0_count-1)

In [118]:
dt_user_ag.head()

,browser,version,systemInfo,platform,platformDetails,extensions,browser_lvl_0,browser_lvl_1,system_info_lvl_0_mean,system_info_lvl_0_max,system_info_lvl_0_min,system_info_lvl_0_count,system_info_lvl_1_mean,system_info_lvl_1_max,system_info_lvl_1_min
0,mozilla,5.0,windows; u; windows nt 5.1; en-us,applewebkit/534.10,"khtml, like gecko",chrome/8.0.552.237 safari/534.10,0.876234,0.596165,0.065151,0.164211,0.028312,4,0.854134,1.000000,0.416538
1,mozilla,5.0,windows nt 5.1; rv:2.0.1,gecko/20100101,not_available,firefox/4.0.1,0.876234,0.596165,0.116152,0.164211,0.068092,2,0.263303,0.416538,0.110068
2,mozilla,5.0,ipod; u; cpu iphone os 4_1 like mac os x; en-us,applewebkit/532.9,"khtml, like gecko",version/4.0.5 mobile/8b117 safari/6531.22.7,0.876234,0.596165,0.018047,0.038910,0.000538,4,1.000000,1.000000,1.000000
3,mozilla,5.0,windows nt 5.1; rv:5.0,gecko/20100101,not_available,firefox/5.0,0.876234,0.596165,0.116152,0.164211,0.068092,2,0.365545,0.416538,0.314552
4,not_available,not_available,not_available,not_available,not_available,not_available,0.099539,1.000000,0.021015,0.021015,0.021015,0,1.000000,1.000000,1.000000


> * We could add more qualitative information about the correlation between multiple infos, eg frquency of the entire/partial tuple

### Platform
It is the same case as the browser and its version. We need then to format it as is and proceed as we did for the latter.

In [122]:
# Extract platform infos
pat_platform = re.compile('(?P<name>.*)\/(?P<version>.*)')
platform_info = dt_user_ag.platform.str.extract(pat_platform,expand = True).fillna('not_available')
# Mapping dicts
platform_info_lvl_0 = platform_info.name.value_counts(normalize = True).to_dict()
platform_info_lvl_1 = platform_info.groupby('name')['version'].value_counts(normalize = True).to_dict()
# Map values
dt_user_ag['platform_lvl_0'] = platform_info.loc[:,'name'].map(platform_info_lvl_0)
dt_user_ag['platform_lvl_1'] = platform_info.loc[:,['name','version']].apply(lambda x: map_tuple(x,platform_info_lvl_1,('name','version')),axis = 1)

### Platform details

In [136]:
dt_user_ag.platformDetails.unique()

array(['khtml, like gecko', 'not_available',
       'khtml, like gecko) version/3.0.4 mobile safari/523.12.2 (admob-android-20101109',
       'khtml, like gecko) (admob-isdk-20100412; iphoneos3.0; j',
       'khtml, like gecko) (admob-isdk-20100614; iphoneos4.2',
       'khtml, like gecko) (admob-isdk-20100614; iphoneos4.1; j',
       'khtml, like gecko) nokia ovi suite/3.1.0.91 (rm-469',
       'khtml, like gecko) (admob-isdk-20101108; iphoneos4.2; j',
       'khtml, like gecko) (admob-isdk-20100908; iphoneos4.1; j',
       'khtml, like gecko) version/3.0.4 mobile safari/523.12.2 (admob-android-20101012',
       'khtml, like gecko) nokia ovi suite/3.1.1.80 (rm-505',
       'khtml, like gecko) (admob-isdk-20101108; iphoneos4.2',
       'khtml, like gecko) (admob-isdk-20100602; iphoneos4.3; j',
       'khtml, like gecko) (admob-isdk-20100908; iphoneos4.2; j',
       'khtml, like gecko) (admob-isdk-20100614; iphoneos4.1; j; c',
       'khtml, like gecko) (admob-isdk-20100614; iphoneos4.2

In [146]:
dt_user_ag.loc[dt_user_ag.platformDetails == 'r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3.5.30729; .net clr 3.0.30729; media center pc 6.0; .net4.0c; .net4.0e; infopath.3']

,browser,version,systemInfo,platform,platformDetails,extensions,browser_lvl_0,browser_lvl_1,system_info_lvl_0_mean,system_info_lvl_0_max,system_info_lvl_0_min,system_info_lvl_0_count,system_info_lvl_1_mean,system_info_lvl_1_max,system_info_lvl_1_min,platform_lvl_0,platform_lvl_1
578506,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
608588,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
608659,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
608801,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
614201,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
651290,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
652206,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
655213,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
655918,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0
667490,mozilla,4.0,compatible; msie 8.0; windows nt 6.1; wow64; t...,;,r1 1.6); slcc2; .net clr 2.0.50727; .net clr 3...,not_available,0.876234,0.403596,0.056222,0.164211,0.000633,10,0.877456,1.0,0.416538,0.510898,1.0


In [147]:
series_user_ag[915931]

'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; GTB7.1; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; (R1 1.6); SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; InfoPath.3)'

### Complements
Besides adding the information about the frequency, we should add a qualitative information regarding the absence of information (available/not_available) for each component of the user-agent

In [ ]:
(?P<browser>\w+)\/(?P<version>[0-9\.]*)(?P<systemInfo> \(.*?\))?(?P<platform> [^(]*(?P<platformDetails> \([^()]*?\))??)?(?P<platform2> [^(]*(?P<platformDetails2> \([^()]*?\))??)?(?P<extensions> [^(]*)?$